In [14]:
import pandas as pd

In [2]:
train_data = pd.read_pickle("train_data.p")

In [70]:
train_data.shape

(926372, 10)

In [3]:
train_avito_data = pd.read_csv("data/train.csv")

In [4]:
fraud = train_avito_data[train_avito_data.index.isin(train_data[train_data["is_bad"]==1].index)]\
                [["title", "description"]].copy()

In [14]:
fraud["list_of_words_title"] = fraud["title"].str.split(" ")

- эвристический подход расчета вероятности при наличии слов тригеров

## title

In [25]:
from collections import Counter

counter = Counter()

fraud['title'].apply(lambda x: counter.update(x.split(" ")))

2         None
5         None
6         None
10        None
12        None
          ... 
984461    None
984477    None
984480    None
984482    None
984486    None
Name: title, Length: 211050, dtype: object

In [29]:
words_triggers_title = ["эт.", "м²", "квартир", "сот.", "участк", "участок", "дом",'iphone',
'6-к', '3-к', '2-к', '4-к', "ваз", "lada",'(ижс)', '5-к', '1-к', 'по','дача', "комната", "гараж", 
"коляска", "шины", "priora"]

In [48]:
df_words_triggers_title = pd.DataFrame(counter.most_common(100))

df_words_triggers_title = df_words_triggers_title[df_words_triggers_title[0]\
                                                  .str.lower().isin(words_triggers_title)]

df_words_triggers_title[1] = df_words_triggers_title[1]/ \
                             df_words_triggers_title[1].sum()*100

In [65]:
fraud["title_proba_fraud"] = fraud["title"].str.lower().apply(lambda x: \
                    df_words_triggers_title[df_words_triggers_title[0].isin(x.split(" "))][1].sum())

## description

In [59]:
from collections import Counter

counter = Counter()

fraud['description'].apply(lambda x: counter.update(x.split(" ")))

2         None
5         None
6         None
10        None
12        None
          ... 
984461    None
984477    None
984480    None
984482    None
984486    None
Name: description, Length: 211050, dtype: object

In [63]:
words_triggers_description = ['телефон', "7", '8', "9", 'вопросы', 'звон', 'номер', 'вопросам','связи'
'↓', '⇒', '+', 'можете', 'сайте', "заказ", "+7", "России", "Москве","тел",
"пишите", "доставка", "обращаться", "ежедневно", "уточняйте", "доставк", "для",
"писать", "чтобы", "сообщения", "напрямую", "компания", "обращайтесь", "почт",
"WhatsApp"]

In [64]:
df_words_triggers_description = pd.DataFrame(counter.most_common(500))

df_words_triggers_description = df_words_triggers_description[df_words_triggers_description[0]\
                                                  .str.lower().isin(words_triggers_description)]

df_words_triggers_description[1] = df_words_triggers_description[1]/ \
                             df_words_triggers_description[1].sum()*100

In [ ]:
fraud["desc_proba_fraud"] = fraud["description"].str.lower().apply(lambda x: \
                    df_words_triggers_description[df_words_triggers_description[0].isin(x.split(" "))][1].sum())

# тесты

- тест 1, попробовать стандартизировать категориальные данные 

In [299]:
X_train.columns

Index(['subcategory', 'price', 'region', 'city', 'description_len',
       'title_len', 'number_of_words_description', 'number_of_words_title',
       'number_of_sentences_description', 'title_proba_fraud',
       'desc_proba_fraud'],
      dtype='object')

In [320]:
to_norm_cols = ['subcategory', 'region', 'city', 'description_len',
       'title_len', 'number_of_words_description', 'number_of_words_title',
       'number_of_sentences_description', 'title_proba_fraud',
       'desc_proba_fraud']

In [321]:
toy_1 = X_train.copy() 
toy_2 = X_val.copy()

In [322]:
for col in to_norm_cols:
    toy_1[col] = (toy_1[col]-toy_1[col].min())/(toy_1[col].max()-toy_1[col].min())
    toy_2[col] = (toy_2[col]-toy_2[col].min())/(toy_2[col].max()-toy_2[col].min())

рез-т - показатель точности не улучшился

- тест 2, попробовал выкинуть аутлаеров по моим показ-м

In [ ]:
toy_1 = train_data.copy()

cols_outl = ['description_len',
       'title_len', 'number_of_words_description', 'number_of_words_title',
       'number_of_sentences_description', 'title_proba_fraud',
       'desc_proba_fraud']

for col in cols_outl:
    toy_1 = toy_1[np.abs(toy_1[col]-toy_1[col].mean()) <= (3*toy_1[col].std())]

    
X_toy_1 = toy_1.drop("is_bad", axis=1).copy()
y_toy_1 = toy_1.is_bad.copy()

# изучение валидационной выборки

In [2]:
import pandas as pd

In [5]:
val = pd.read_csv(r"data/val.csv")

In [9]:
for text in val[val["is_bad"]==1]["description"].head(100):
    display(text)

'Звонить 89425546881'

'Шины зимние б/у Marshal Assimetric I”Zen KW 61 681/41 r 63 диски AVA  литые универсальные 4 J 63 3/600 или 3/663,2 ЕТ 28~31. Забрать можно на Ярославском шоссе у МКАД или в г. Сергиев Посад. Могу подвезти(по договорённости) Состояние- на 6~5 сезона. БЕЗ ТОРГА и. 8 964 688 36 68'

'Продам авто в хорошем состоянии.все вопросы по тел.Двигатель стоит родной,611л/с ,самый надёжный в этой линейке.Салон фестфалия-трансформер,очень редкая комплектация:диван раскладывается и получаеться двухспальная кровать,выдвижной столик,монитор,люк,электро жабры,электро стеклоподъемники,электро зеркала с подогревом,кожаный салон,передняя подушка безопасности,потолок алькантара,кондиционер,вебаста с таймером,очень удобно зимой.ABS,ESP,ASR,блокировка передних колёс (W),Теперь из не достатков:присутствует не большая коррозия в углу на правом пороге,притертость не значительная на сдвижной двери.От машины не избавляюсь,езжу каждый день,так что перекупам и стоянки просьба не беспокоить.Торг приветствуется,в подарок отдам зимнюю резину на литых дисках.звоните 8 девятьсот девятнадцать694 тридцать четыре 29 .'

'toyo observe g5-ice   без грыж и порезов звонить на( 89155903052)'

'Продаётся 5 комнатная квартира, косметический ремонт! Чистая не прокуренная квартира без животных!) Остается кухонный гарнитур, кондиционер, в ванной навесной шкаф и стиральная машинка! облагороженый двор, в шаговой доступности остановка, магазин магнит, оптовая база ТРАМ! реальному покупателю торг при осмотре. Телефон: 8ш9а3ш8ш0ш6ш9ш0ш1ш5ш8'

...

'Продам или обменяю на 543х40, 523х43, 523х30 р18. если не доступен звонить на восемь девятсот824638655. торг'

'!!! 8-932-207-181-46 временно звонить на этот номер!!!/\n квартиры посуточно и на длительный срок со скидкой/\n            ! ГАРАНТИЯ ЧИСТОТЫ ИЛИ ВЕРНЕМ ДЕНЬГИ!!!/\n            ! рядом Планета, Арена Север, Ванкорнефть/\n            ! TV,VI-FI/\n            ! ВКУСНЫЕ ПРЕЗЕНТЫ/\n            ! необходимые ср-ва гигиены/\n            !полноценная кухня: посуда, СВЧ, чайник, холодильник/\n            !всегда ЧИСТЫЕ подушки, одеяла, полотенца, белье/\n                  ДЛЯ ШУМНЫХ ВЕЧЕРИНОК НЕ ПОДХОДИТ!'

'Найден той-терьер в районе Шевченко. Бежал за нами до самого дома. Приютили до вечера. Ищем только старых хозяев!/\n Звонить по телефону 30-41-94'

'!!! -965-507-484-31 временно звонить на этот номер!!!/\n квартиры посуточно и на длительный срок со скидкой/\n            ! ГАРАНТИЯ ЧИСТОТЫ ИЛИ ВЕРНЕМ ДЕНЬГИ!!!/\n            ! рядом Планета, Арена Север, Ванкорнефть/\n            ! TV,VI-FI/\n            ! ВКУСНЫЕ ПРЕЗЕНТЫ/\n            ! необходимые ср-ва гигиены/\n            !полноценная кухня: посуда, СВЧ, чайник, холодильник/\n            !всегда ЧИСТЫЕ подушки, одеяла, полотенца, белье/\n                  ДЛЯ ШУМНЫХ ВЕЧЕРИНОК НЕ ПОДХОДИТ!'

'Продам пригласительную ссылку на сайт накрутки для Инстаграм 4200₽/\n/\nТакие ссылки стоят в разы дороже /\n/\nМожно накручивать подписчиков , просмотры, комментарии, лайки , охваты , статистику /\nСайт nakrutka . by пользуюсь уже 5 года, сайт постоянно обновляет цены, делает скидки и выгодные предложения /\nНа сайте есть опция бесплатная накрутка , раз в сутки /\nЦены указываются за 4000 , а не за штуку /\n/\nТак же вы можете потом продавать вашу ссылку другим людям'

In [14]:
from collections import Counter

counter = Counter()

val['description'].apply(lambda x: counter.update(x.split(" ")))
    
words_triggers_description = ['телефон', "7", '8', "9", 'вопросы', 'звон', 'номер', 'вопросам','связи'
    '↓', '⇒', '+', 'можете', 'сайт', "заказ", "россии", "москве","тел",
    "пишите", "доставка", "обращаться", "ежедневно", "уточняйте", "доставк", "для",
    "писать", "чтобы", "сообщен", "напрям", "компания", "обращайтесь", "почт",
    "whatsApp", "запч", "наличии", "работ", "выбор", "наш", "сообщите", "может", "артикул", 
    "продам"]
    
counter.most_common(100)


[('', 34525),
 ('в', 29572),
 ('и', 27132),
 ('на', 23410),
 ('-', 21128),
 ('с', 15233),
 ('по', 12511),
 ('для', 9409),
 ('до', 6536),
 ('не', 6174),
 ('от', 5920),
 ('или', 4253),
 ('В', 4229),
 ('—', 3579),
 ('запчастей', 3412),
 ('из', 3085),
 ('за', 2968),
 ('без', 2954),
 ('все', 2889),
 (',', 2837),
 ('можно', 2610),
 ('/', 2596),
 ('при', 2494),
 ('есть', 2474),
 ('4', 2088),
 ('6', 2066),
 ('5', 2038),
 ('3', 2014),
 ('к', 1994),
 ('1', 1985),
 ('2', 1970),
 ('под', 1879),
 ('у', 1843),
 ('/\n', 1832),
 ('–', 1790),
 ('наличии', 1783),
 ('работы', 1757),
 ('сайте', 1620),
 ('России', 1614),
 ('выбор', 1591),
 ('9', 1589),
 ('нашем', 1587),
 ('/\n-', 1567),
 (':', 1544),
 ('любую', 1464),
 ('а', 1342),
 ('же', 1318),
 ('как', 1280),
 ('+', 1239),
 ('Продам', 1232),
 ('этом', 1222),
 ('номер', 1186),
 ('запчасти', 1170),
 ('любой', 1158),
 ('Большой', 1154),
 ('более', 1145),
 ('больше', 1136),
 ('чем', 1111),
 ('нашей', 1104),
 ('г.', 1102),
 ('так', 1093),
 ('ваши', 1091),
 (

In [44]:
a = pd.DataFrame(["123 asdfafads asd", "asdf dsddsdssdds asdf"])

In [45]:
a

,0
0,123 asdfafads asd
1,asdf dsddsdssdds asdf


In [46]:
a[0].apply(lambda x: sum(len(word) for word in str(x).split(" ")) \
                                                     / len(str(x).split(" ")))

0    5.000000
1    6.333333
Name: 0, dtype: float64

In [47]:
a[0].apply(lambda x: sum(len(word) for word in str(x).split(" ")) )

0    15
1    19
Name: 0, dtype: int64

In [48]:
a[0].apply(lambda x: len(str(x).split(" ")))

0    3
1    3
Name: 0, dtype: int64